## 5. Plummer sphere

Now let's set up a system with lots of particles! The simplest system we can imagine is the ergodic Plummer sphere we derived in class.

In [ ]:
# Create ICs for a Plummer sphere with characteristic scale 1pc, mass 1e6 Msun, sampled with 5000 particles
Plummer_IC = IC.Plummer(b=1*u.pc, totmass=1e6*u.Msun, N=5000)

In [ ]:
# Create a simulation with timesteps of 0.005 Myr and softening of 0.05 pc
sim_P = Simulation(dt=0.005*u.Myr, eps=0.05*u.pc)
# Add the Plummer ICs
sim_P.add_IC(Plummer_IC)

In [ ]:
# Run for 500 time steps
sim_P.run(500)

We can plot the particle distribution projected along different axes using the ``plot_particles()`` method. We'll start by looking at the initial particle positions (``snap='IC'``):

In [ ]:
fig = plt.figure(figsize=(8,9))
ax1 = fig.add_subplot(221, aspect=1.0)
ax2 = fig.add_subplot(222, aspect=1.0)
ax3 = fig.add_subplot(223, aspect=1.0)

# Plot the x-y projection of positions onto axis ax1 in units of pc with limits +/-5
sim_P.plot_particles(snap='IC', parm='pos', coords='xy',  unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax1)
# ...and the other projections
sim_P.plot_particles(snap='IC', parm='pos', coords='zy', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax2)
sim_P.plot_particles(snap='IC', parm='pos', coords='xz', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax3)

We can compare this to the final snapshot by specifying ``snap='final'``. Other options are a specific snapshot number.

In [ ]:
fig = plt.figure(figsize=(8,9))
ax1 = fig.add_subplot(221, aspect=1.0)
ax2 = fig.add_subplot(222, aspect=1.0)
ax3 = fig.add_subplot(223, aspect=1.0)

sim_P.plot_particles(snap='final', parm='pos', coords='xy',  unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax1)
sim_P.plot_particles(snap='final', parm='pos', coords='zy', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax2)
sim_P.plot_particles(snap='final', parm='pos', coords='xz', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax3)

Look at these! Does the distribution appear to be in equilibrium?

We can examine other aspects of the simulation too, such as the velocities.
**FIXME:** Use ``parm='vel'`` to compare the initial and final velocity ellipsoid (note: you should probably eliminate or change the ``xlim`` and ``ylim`` parameters. What do you think?

Let's make a movie of the positions and velocities! This will save the movie to ``Plummer_posvel.mp4``; you can change this in the ``anim.save()`` line as you make other movies.

In [ ]:
fig = plt.figure(figsize=(15,7))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121, aspect=1.0)
    ax2 = fig.add_subplot(122, aspect=1.0)
    
    # First plot: particle positions xy in units of pc
    posplot = sim_P.plot_particles(snap=frame, parm='pos', coords='xy', unit=u.pc, \
                                 xlim=[-5,5], ylim=[-5,5], ax=ax1)
    # Second plot: velocity ellipsoid xy in units of km/s
    velplot = sim_P.plot_particles(snap=frame, parm='vel', coords='xy', unit=u.km/u.s, \
                                xlim=[-75,75], ylim=[-75,75], ax=ax2)
    return posplot, velplot

ms_per_frame = 40   # 25 frames per second
anim = FuncAnimation(fig, animate, frames=sim_P.Nsnap, interval=ms_per_frame)
anim.save('Plummer_posvel.mp4')

plt.close(fig)

What do you think of them? What do you think about how long it takes to do the simulation vs. to create the movie?

We can also turn a GravHopper snapshot into a pynbody snapshot using the ``pyn_snap()`` function, so we can use pynbody's analysis routines to create profiles. So let's make a movie that shows the density profile and velocity dispersion profile over the course of the simulation.

In [ ]:
fig = plt.figure(figsize=(20,8))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # Create a pynbody snapshot of the given snapshot
    snap = sim_P.pyn_snap(frame)
    # The conversion routine forces position units to kpc, but pc is much more convenient
    # for this situation, so convert it back before making the profile.
    snap['pos'].convert_units('pc')
    # Make a 3D profile of the snapshot
    prof = profile.Profile(snap, ndim=3, nbins=200)
    
    # First plot: density profile
    densplot = ax1.plot(prof['rbins'], prof['density'])
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlim(1, 50)
    ax1.set_ylim(1e-4,3e4)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())
    ax1.set_xlabel(radial_label)
    ax1.set_ylabel('$\\rho$ (${0}$)'.format(prof['density'].units.latex()))
    ax1.set_title('{0:.2f}'.format(sim_P.times[frame]))
    
    # Second plot: radial velocity dispersion profile
    sigmaplot = ax2.plot(prof['rbins'], prof['vr_disp'].in_units('km s^-1'))
    ax2.set_xscale('log')
    ax2.set_xlim(1, 50)
    ax2.set_ylim(0, 25)
    ax2.set_xlabel(radial_label)
    ax2.set_ylabel('$\\sigma_r$ (km/s)')
    
    return densplot, sigmaplot

anim = FuncAnimation(fig, animate, frames=sim_P.Nsnap, interval=ms_per_frame)
anim.save('Plummer_profiles.mp4')

plt.close(fig)

What do you think of these? How does the $\sigma_r$ plot relate to the particle velocity plots earlier? Do you expect the simulation to be in equilibrium? Does it appear to be based on these various plots and movies? How do you think I decided what value of ``dt`` to use for the simulation?

## 6. Play with the Plummer sphere

**FIXME:** Try changing the scale length ``b`` of the Plummer sphere (you will probably need to play with the limits of the various axes). How would you expect the velocity dispersion to scale with $b$ based on analytic arguments? How does that compare to what you see?

**FIXME:** Try changing the total mass of the Plummer sphere. How would you expect the velocity dispersion to scale with mass? How does that compare to what you see?

## 7. Truncated Singular Isothermal Sphere

We investigated the Truncated Singular Isothermal Sphere analytically in Assignment 4 -- now let's examine it numerically!

In [ ]:
# Create a TSIS with a mass of 1e11 Msun, a cutoff radius of 100 kpc, sampled with 5000 particles
TSIS_IC = IC.TSIS(totmass=1e11*u.Msun, maxrad=100*u.kpc, N=5000)

**FIXME:** Create a simulation with a time step of 5 Myr, softening of 0.2 kpc. Add the TSIS ICs, and run it for 500 time steps.

In [ ]:
sim_TSIS = #FIXME
sim_TSIS.add_IC(#FIXME
sim_TSIS.run(#FIXME

Let's make movies of the positions/velocities and the density/velocity dispersion profiles for this case too.

In [ ]:
fig = plt.figure(figsize=(10,5))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121, aspect=1.0)
    ax2 = fig.add_subplot(122, aspect=1.0)
    
    # First plot: particle positions xy in units of pc
    posplot = sim_TSIS.plot_particles(snap=frame, parm='pos', coords='xy', unit=u.kpc, \
                                 xlim=[-200,200], ylim=[-200,200], ax=ax1)
    # Second plot: velocity ellipsoid xy in units of km/s
    velplot = sim_TSIS.plot_particles(snap=frame, parm='vel', coords='xy', unit=u.km/u.s, \
                                xlim=[-200,200], ylim=[-200,200], ax=ax2)
    return posplot, velplot

ms_per_frame = 40   # 25 frames per second
anim = FuncAnimation(fig, animate, frames=sim_TSIS.Nsnap, interval=ms_per_frame)
anim.save('TSIS_posvel.mp4')

plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(20,8))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # Create a pynbody snapshot of the given snapshot and make a 3D profile from it
    snap = sim_TSIS.pyn_snap(frame)
    # Make a 3D profile of the snapshot
    prof = profile.Profile(snap, ndim=3, nbins=200)
    
    # First plot: density profile
    densplot = ax1.plot(prof['rbins'], prof['density'])
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlim(0.2, 400)
    ax1.set_ylim(1e-1,1e10)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())
    ax1.set_xlabel(radial_label)
    ax1.set_ylabel('$\\rho$ (${0}$)'.format(prof['density'].units.latex()))
    ax1.set_title('{0:.2f}'.format(sim_TSIS.times[frame]))
    
    # Second plot: radial velocity dispersion profile
    sigmaplot = ax2.plot(prof['rbins'], prof['vr_disp'].in_units('km s^-1'))
    ax2.set_xscale('log')
    ax2.set_xlim(0.2, 400)
    ax2.set_ylim(0, 60)
    ax2.set_xlabel(radial_label)
    ax2.set_ylabel('$\\sigma_r$ (km/s)')
    
    return densplot, sigmaplot

anim = FuncAnimation(fig, animate, frames=sim_TSIS.Nsnap, interval=ms_per_frame)
anim.save('TSIS_profiles.mp4')

plt.close(fig)

How does this situation differ from the Plummer sphere? What happens at small radius? What happens near the truncation radius? What happens at large radius? Think about the TSIS question from Assignment 4 -- what would you have expected to happen based on your analysis? How does that relate to what you see here? Think about Liouville's Theorem -- how is what you see in the ``TSIS_posvel.mp4`` movie an example of that?

## 8. Do something interesting

Modify something in one of the spherical examples. For example:
 - Try changing ``dt`` significantly. What effect does it have?
 - Try changing ``N`` significantly. How does it affect what you see? How does it affect how long it takes the simulation to run?
 - Try running the simulation for much longer. Do you see any distinct long-term evolution of the system that's different from what happens on shorter timescales?
 - Use the Hernquist sphere (you can run ``help(IC.Hernquist)`` to find the parameters it takes). How does it behave compared to the other examples?
 - Use one of galpy's other spherical distribution functions (see [here](https://docs.galpy.org/en/latest/reference/df.html)) and ``IC.from_galpy_df()`` to do another IC. Note that the galpy DFs don't play nicely with astropy units, so you will need to convert the units manually.